In [51]:
from time import sleep
import requests
import pandas as pd
# import matplotlib.pyplot as plt
from config import api_key
import datetime as dt
import json
from pprint import pprint
import pymongo
from pymongo import MongoClient

In [52]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [53]:
db = client["stocktests2"]

In [54]:
collection = db["stock_collection3"]

In [55]:
#create a function to generate a list of all tickers
sp_500 = pd.read_csv("../Datasets/constituents_csv.csv")
sp_500.count()
ticker_symbol = sp_500['Symbol']



In [56]:
ticker_list =[]
for x in range(len(sp_500)):
#     sector_name = sp_500['Sector'][x]
#     sector_id = session.query(Sector.sector_id).filter(Sector.sector == sector_name).one()[0]
    ticker_list.append(sp_500['Symbol'][x])


In [57]:
ticker_list
ticker_list_test = ticker_list[0:10]
ticker_list_test

['AMZN', 'BBY', 'KMX', 'DISCA', 'EXPE', 'GRMN', 'M', 'MAR', 'MCD', 'NKE']

In [58]:
base_url = "https://www.alphavantage.co/query?"
def alpha_stock_request(function,symbol):
    return requests.get(base_url + f"function={function}&symbol={symbol}&outputsize=full&apikey={api_key}")

# def alpha_currency_request(function,from_symbol,to_symbol):
#     return requests.get(base_url + f"function={function}&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}")

In [59]:
# create compoany documents
company_documents = []

for i in ticker_list_test:
    company_documents.append([i])


In [60]:
print(company_documents)

[['AMZN'], ['BBY'], ['KMX'], ['DISCA'], ['EXPE'], ['GRMN'], ['M'], ['MAR'], ['MCD'], ['NKE']]


In [61]:
# ticker_list = db.stocksTest.find()
# ticker_lista = [x[0] for x in ticker_list]
list_of_dicts = []

for ticker in ticker_list_test:

        sleep(3)
        try:
            results = alpha_stock_request("TIME_SERIES_DAILY",ticker).json()
            today = results['Meta Data']['3. Last Refreshed']
            data = results['Time Series (Daily)']

            for x in data:
                if x != today: #get yesterday and before data (exclude today)
#                     year , month , day = x.split("-")
#                     date_ref = year + month + day
                    row = data[x]
                    open_price = row['1. open']
                    close_price = row['4. close']
                    high_price = row['2. high']
                    low_price = row['3. low']
                    volume = row['5. volume']
                    list_of_dicts.append(
                          {
                            "ticker": ticker,
                            "date":x, 
                            "open_price" : open_price,
                            "close_price" : close_price,
                            "high_price" : high_price,
                            "low_price" : low_price,
                            "volume" : volume
                                    }
                                )

#                     new_stock = {"date": x, "open_price" : open_price, "close_price" : close_price,
#                                  "high_price" : high_price,
#                                  "low_price" : low_price,
#                                  "volume" : volume}
#                     list_of_dicts.append(
#                         {
#                         "ticker": ticker,
#                         "data" : new_stock)
#                       }
#                     )

 


        except KeyError:
            print(f"Company {ticker} Not Found")

In [ ]:
pprint(list_of_dicts)

In [62]:
def insert_stocks(list_of_dicts):
#     db.stock_collection.drop()
    document = collection.insert_many(list_of_dicts)
#     document = collection.insert_one(list_of_dicts)
#     document.acknowledged

In [63]:
inser_data_mongodb = insert_stocks(list_of_dicts)

In [95]:
print(iddd)

None


In [64]:
client.close()

In [65]:
list_of_dicts



[{'ticker': 'AMZN',
  'date': '2020-04-02',
  'open_price': '1901.6400',
  'close_price': '1918.8300',
  'high_price': '1927.5300',
  'low_price': '1890.0000',
  'volume': '4335985',
  '_id': ObjectId('5e89efaf3a046a78bafae84e')},
 {'ticker': 'AMZN',
  'date': '2020-04-01',
  'open_price': '1932.9700',
  'close_price': '1907.7000',
  'high_price': '1944.9600',
  'low_price': '1893.0000',
  'volume': '4121875',
  '_id': ObjectId('5e89efaf3a046a78bafae84f')},
 {'ticker': 'AMZN',
  'date': '2020-03-31',
  'open_price': '1964.3500',
  'close_price': '1949.7200',
  'high_price': '1993.0200',
  'low_price': '1944.0100',
  'volume': '5123626',
  '_id': ObjectId('5e89efaf3a046a78bafae850')},
 {'ticker': 'AMZN',
  'date': '2020-03-30',
  'open_price': '1922.8300',
  'close_price': '1963.9500',
  'high_price': '1973.6300',
  'low_price': '1912.3400',
  'volume': '6126087',
  '_id': ObjectId('5e89efaf3a046a78bafae851')},
 {'ticker': 'AMZN',
  'date': '2020-03-27',
  'open_price': '1930.8600',
  '